In [1]:
import gmaps
import json
import requests
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plt
from scipy import stats
from config import api_key
from citipy import citipy

In [142]:
abnb_data = pd.read_csv("Resources/airbnb_data.csv")

/Users/alfonsocamero/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [188]:
abnb2 = abnb_data.drop(['neighbourhood_group', 'id', 'host_id', 'host_name', 'calculated_host_listings_count', 'last_review'], axis=1)
abnb2_df = abnb2.drop(abnb2[abnb2['room_type'] == 'Private room'].index)
abnb2_df['city'] = abnb2_df['city'].replace({'Broward County':'Fort Lauderdale','Clark County':'Las Vegas','New York City':'New York','San Clara Country':'San Jose','San Mateo County':'San Mateo','Santa Cruz County':'Santa Cruz','Twin Cities MSA':'Twin Cities','Washington D.C.':'Washington'})
abnb_df = abnb2_df.drop(abnb2_df[(abnb2_df['city'] == 'Hawaii') | (abnb2_df['city'] == 'Rhode Island') | (abnb2_df['city'] == 'Twin Cities')].index)

abnb_df = abnb_df[['city', 'name', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']]
abnb_df.rename(columns={'city': 'City', 'name': 'Property Name', 'neighbourhood': 'Neighborhood', 'latitude': 'Latitude', 'longitude': 'Longitude', 'room_type': 'Room Type', 'price': 'Price', 'minimum_nights': 'Minimum Nights', 'number_of_reviews': 'Number of Reviews', 'reviews_per_month': 'Reviews per Month', 'availability_365': 'Availability (days per year)'}, inplace=True)

abnb_df


,City,Property Name,Neighborhood,Latitude,Longitude,Room Type,Price,Minimum Nights,Number of Reviews,Reviews per Month,Availability (days per year)
1,Asheville,French Chic Loft,28801,35.597790,-82.555400,Entire home/apt,470,1,114,1.03,288
2,Asheville,Walk to stores/parks/downtown. Fenced yard/Pet...,28801,35.606700,-82.555630,Entire home/apt,75,30,89,0.81,298
3,Asheville,Cottage! BonPaul + Sharky's Hostel,28806,35.578640,-82.595780,Entire home/apt,90,1,267,2.39,0
5,Asheville,Terrace Cottage,28804,35.618560,-82.552760,Entire home/apt,134,7,54,0.49,294
8,Asheville,"Private, peaceful, and free goat therapy",28805,35.619290,-82.481140,Entire home/apt,71,28,537,5.01,207
...,...,...,...,...,...,...,...,...,...,...,...
226025,Washington,"DC Hidden In Plain ""Site""","Downtown, Chinatown, Penn Quarters, Mount Vern...",38.903880,-77.029730,Entire home/apt,104,1,0,NaN,99
226026,Washington,DC 3 BR w/ screen porch 3 blck to metro w/ par...,"Brookland, Brentwood, Langdon",38.920820,-76.990980,Entire home/apt,151,2,0,NaN,300
226027,Washington,Charming Penthouse Apt w/ Rooftop Terrace in L...,"Shaw, Logan Circle",38.911170,-77.033540,Entire home/apt,240,2,0,NaN,173
226028,Washington,Adams Morgan/Nat'l Zoo 1 BR Apt #32,"Kalorama Heights, Adams Morgan, Lanier Heights",38.926630,-77.044360,Entire home/apt,60,21,0,NaN,362


In [178]:
Cities = []
for city in abnb_df['City']:
    if city not in Cities:
        Cities.append(city)

In [179]:
cities = [city.lower() for city in Cities]

In [180]:
states = ['NC', 'TX', 'MA', 'FL', 'MA', 'IL', 'NV', 'OH', 'CO', 'NJ', 'CA', 'TN', 'LA', 'NY', 'CA', 'CA', 'OR', 'OR', 'CA', 'CA', 'CA', 'CA', 'CA', 'WA', 'DC']

In [181]:
loop_df = pd.DataFrame({
    'city': cities,
    'City': Cities,
    'State': states
})

loop_df['Population'] = ''

loop_df

,city,City,State,Population
0,asheville,Asheville,NC,
1,austin,Austin,TX,
2,boston,Boston,MA,
3,fort lauderdale,Fort Lauderdale,FL,
4,cambridge,Cambridge,MA,
5,chicago,Chicago,IL,
6,las vegas,Las Vegas,NV,
7,columbus,Columbus,OH,
8,denver,Denver,CO,
9,jersey city,Jersey City,NJ,


In [182]:
for index, row in loop_df.iterrows():
    city = row["city"]
    City = row["City"]
    state = row["State"]
    url = f'http://public.opendatasoft.com/api/records/1.0/search/?dataset=cities-and-towns-of-the-united-states%40public&q={city}&facet=name&facet=state&refine.name={City}&refine.state={state}'
    response = requests.get(url).json()
    print(f"Searching population record for {city}")
    
    try:
        loop_df.loc[index, "Population"] = response['records'][0]['fields']['pop_2010']
    
        print(f"{city}'s population: {response['records'][0]['fields']['pop_2010']}")
    
        print('________________________________________________')
    
    except (KeyError, IndexError):
        print(f"Population for {city} not found... Continue scanning.")
        
        print('________________________________________________')

Searching population record for asheville
asheville's population: 83393
________________________________________________
Searching population record for austin
austin's population: 790390
________________________________________________
Searching population record for boston
boston's population: 617594
________________________________________________
Searching population record for fort lauderdale
fort lauderdale's population: 165521
________________________________________________
Searching population record for cambridge
cambridge's population: 105162
________________________________________________
Searching population record for chicago
chicago's population: 2695598
________________________________________________
Searching population record for las vegas
las vegas's population: 583756
________________________________________________
Searching population record for columbus
columbus's population: 787033
________________________________________________
Searching population record fo

In [184]:
loop_df

,city,City,State,Population
0,asheville,Asheville,NC,83393
1,austin,Austin,TX,790390
2,boston,Boston,MA,617594
3,fort lauderdale,Fort Lauderdale,FL,165521
4,cambridge,Cambridge,MA,105162
5,chicago,Chicago,IL,2695598
6,las vegas,Las Vegas,NV,583756
7,columbus,Columbus,OH,787033
8,denver,Denver,CO,600158
9,jersey city,Jersey City,NJ,247597


In [190]:
graphs_draft_df = abnb_df.groupby('City').mean().reset_index()
graphs_draft_df['Population'] = loop_df['Population']
graphs_draft_df['State'] = loop_df['State']
graphs_df = graphs_draft_df[['City', 'State', 'Population', 'Latitude', 'Longitude', 'Price', 'Minimum Nights', 'Number of Reviews', 'Reviews per Month', 'Availability (days per year)']]
graphs_df

,City,State,Population,Latitude,Longitude,Price,Minimum Nights,Number of Reviews,Reviews per Month,Availability (days per year)
0,Asheville,NC,83393,35.578463,-82.557080,217.851462,5.445029,76.460819,2.915155,169.638596
1,Austin,TX,790390,30.282802,-97.755954,311.782487,9.479961,33.144644,1.439457,142.824630
2,Boston,MA,617594,42.343169,-71.080080,209.841602,34.919891,37.570778,1.625987,184.854802
3,Cambridge,FL,165521,42.371986,-71.107202,222.803670,39.790826,49.317431,1.513995,136.440367
4,Chicago,MA,105162,41.902314,-87.662216,179.361744,9.127082,42.281770,1.843966,177.314855
5,Columbus,IL,2695598,39.984240,-82.996481,395.042333,8.599247,47.689558,2.868683,160.244591
6,Denver,NV,583756,39.741899,-104.981299,174.750077,12.829403,48.564165,2.132750,129.939663
7,Fort Lauderdale,OH,787033,26.098144,-80.138814,264.226556,7.100666,21.238818,1.339888,202.122153
8,Jersey City,CO,600158,40.726335,-74.052016,167.781026,10.259547,29.986277,1.543465,186.286993
9,Las Vegas,NJ,247597,36.116909,-115.145809,243.013242,7.317966,32.129312,1.863386,184.842406
